In [2]:
#!/usr/bin/env python

import fill_untappd
import yaml, json, sqlite3,  dateparser,os, sys
from pprint import pprint
# from airtable import Airtable
import pandas as pd

In [4]:
conn = sqlite3.connect("./db/untappd.db")
df = pd.read_sql_query("select * from checkins;", conn)
jsons  = df.json.apply(lambda j: json.loads(j))

venues = jsons.apply(
    lambda x: 
    pd.concat([
        pd.Series(x["venue"]),
        pd.Series(x["user"])[["user_name"]],
        pd.Series(x["beer"]),
        pd.Series(x)[["checkin_id","created_at","rating_score"]]
        ])
        )

venues_info = pd.concat([venues, 
pd.concat([
    venues['contact'].apply(lambda x: pd.Series(x)),
    venues['location'].apply(lambda x: pd.Series(x)),
], axis = 1)],axis=1).set_index(["venue_slug","beer_slug"])

venue_users = venues_info.groupby(["venue_slug","beer_slug"]).user_name.apply(','.join).rename("user_names")
venue_rating = venues_info.groupby(["venue_slug","beer_slug"]).rating_score.sum().rename("total_rating")
venue_counts = venues_info.groupby(["venue_slug","beer_slug"]).size().rename("n_checkins").astype(int)
venue_last_checkin = venues_info.groupby(["venue_slug","beer_slug"]).created_at.apply(lambda x: x.apply(lambda y:dateparser.parse(y)).max()).rename("last_checkin")


venues_info_dedup = venues_info.loc[~venues_info.index.duplicated()]
venues_annotated = venues_info_dedup.join(pd.concat([venue_users, venue_rating, venue_counts,venue_last_checkin],axis=1))

venues_annotated = venues_annotated.loc[ lambda x: x.venue_url != ""]
venues_annotated = venues_annotated.loc[ venues_annotated.venue_url.notna()]
venues_annotated = venues_annotated.loc[ lambda x: x.venue_city != ""]
venues_annotated = venues_annotated.loc[ venues_annotated.venue_city.notna()]


cans = pd.read_csv("../_data/cans.csv")

for i, r in cans.iterrows():
    s = yaml.dump(r.to_dict(), default_flow_style=False)
    with open(f"../_cans/{r.code}.md","w") as f:
        f.write("---\n"),
        f.write(s)
        f.write("\n---\n")

cans["untappd_id"] = cans.untappd.str.extract('(\d*$)').astype(int)

venue_beers_annotated = venues_annotated.loc[lambda x: x.venue_id.notna()]
venue_beers_annotated = venue_beers_annotated.join(cans.set_index("untappd_id")[["code"]], on ="bid")
venue_beers_annotated = venue_beers_annotated.rename({"code":"can_code"}, axis = "columns")
venue_beers_annotated["venue_id"] = venue_beers_annotated["venue_id"].astype(int)
venue_beers_annotated.n_checkins = venue_beers_annotated.n_checkins.astype(int)

venue_beers_annotated.to_csv("../_data/venue_beers.csv")

venues_annotated = venue_beers_annotated.reset_index()[["venue_name","venue_slug","primary_category","contact","venue_id","lat","lng","venue_city" , "venue_address","venue_country","venue_city",	"venue_country",	"venue_state",'venue_url']]
venues_annotated.to_csv("../_data/venues.csv")


/var/folders/jb/0c007hxx54g97_0_9vdk4nk00000gn/T/ipykernel_50491/1840757843.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pd.Series(x["venue"]),


In [65]:
jsons

0      {'checkin_id': 953341972, 'created_at': 'Wed, ...
1      {'checkin_id': 953343161, 'created_at': 'Wed, ...
2      {'checkin_id': 953359908, 'created_at': 'Wed, ...
3      {'checkin_id': 953362324, 'created_at': 'Wed, ...
4      {'checkin_id': 953363112, 'created_at': 'Wed, ...
                             ...                        
645    {'checkin_id': 1123179567, 'created_at': 'Sun,...
646    {'checkin_id': 1123203612, 'created_at': 'Sun,...
647    {'checkin_id': 1123850824, 'created_at': 'Thu,...
648    {'checkin_id': 1123853610, 'created_at': 'Thu,...
649    {'checkin_id': 1125017922, 'created_at': 'Sun,...
Name: json, Length: 650, dtype: object

In [67]:
checkins_df = jsons.apply(
    lambda y:pd.Series({"has_photo":
              ('media' in y) and (y['media']['items'] is not None) and (len(y['media']['items']) > 0),
              "checkin_id":y["checkin_id"],
              "created_at":y["created_at"],
             }
)
)

In [68]:
checkins_df


,has_photo,checkin_id,created_at
0,True,953341972,"Wed, 14 Oct 2020 22:12:09 +0000"
1,True,953343161,"Wed, 14 Oct 2020 22:15:43 +0000"
2,False,953359908,"Wed, 14 Oct 2020 23:04:11 +0000"
3,True,953362324,"Wed, 14 Oct 2020 23:10:38 +0000"
4,False,953363112,"Wed, 14 Oct 2020 23:12:38 +0000"
...,...,...,...
645,False,1123179567,"Sun, 23 Jan 2022 19:23:37 +0000"
646,True,1123203612,"Sun, 23 Jan 2022 20:15:30 +0000"
647,False,1123850824,"Thu, 27 Jan 2022 18:50:59 +0000"
648,False,1123853610,"Thu, 27 Jan 2022 19:01:49 +0000"


In [60]:
# fetch checkin images
import wget
checkin_photos = jsons.loc[lambda x: x.apply(lambda y:('media' in y) and (y['media']['items'] is not None) and (len(y['media']['items']) > 0))].apply(lambda x:
                        pd.Series(dict(**{
                            "checkin_id":x["checkin_id"],
                            "photo":x['media']['items'][0]['photo']['photo_img_md'],
                            "photo_sm":x['media']['items'][0]['photo']['photo_img_sm'],
                        }
                                      ))).set_index("checkin_id")





checkin_photos.apply(
        lambda x:wget.download(x.photo,f"../assets/checkin_photos/{x.name}.jpg") if not (os.path.isfile(f"../assets/checkin_photos/{x.name}.jpg")) else None
,axis=1)

checkin_photos.apply(
        lambda x:wget.download(x.photo_sm,f"../assets/checkin_photos/{x.name}_sm.jpg") if not (os.path.isfile(f"../assets/checkin_photos/{x.name}_sm.jpg")) else None
,axis=1)


checkin_id
953341972     None
953343161     None
953362324     None
953373944     None
953375832     None
              ... 
1122693967    None
1122698579    None
1122788026    None
1122789389    None
1123203612    None
Length: 306, dtype: object